In [ ]:
"""Sandbox module."""


from data_generation import (
    gen_discretized_distributions,
    gen_grids_and_parameters,
)

%matplotlib inline

In [ ]:
# Set up data
n = 200
gridnum = 1000
truncation_point = 3

grid_pdfs, grid_qfs, mus, sigmas = gen_grids_and_parameters(
    n, gridnum, truncation_point, delta=0,
)

In [ ]:
# Generate distributions
pdfs_discretized, cdfs_discretized, qfs_discretized, qdfs_discretized = (
    gen_discretized_distributions(grid_pdfs, grid_qfs, mus, sigmas, truncation_point)
)